<br>
<br>

**Population Densities & Missing Years**

<br>

## Setting Up

In [1]:
setwd(base::dirname(getwd()))
getwd()

[1] "J:/library/helminthiases/spatial"

<br>

## Functions

A data frame for each map year

In [2]:
#' @describe For a given year, this function extracts the estimated population density, per longitude & latitude 
#'           pair, from the year's world population dentsity map
#'
#' @param year: The year of interest.
#' @param root: The location of the year's map file.
#' @param affix: The affix of the map file name.
#' @param frame: A data frame consisting of the longitude & latitude points of interest
#' 
temporary <- function (year, root, affix, frame) {

  # map
  path <- file.path(root, paste0(year, affix))
  map <- terra::rast(file.path(path))

  # The longitude & latitude points
  points <- frame[, c('longitude', 'latitude')]

  # Variable values w.r.t. ...
  derivations <- terra::extract(x = map, y = points, method = 'bilinear', xy = TRUE)
  row.names(derivations) <- row.names(frame)

  # Drop the record number field, i.e., drop <ID>
  derivations <- base::subset(x = derivations, select = -ID)

  # The field names
  estimate <- colnames(derivations)[startsWith(colnames(derivations), prefix = 'gpw_v4_')]
  derivations <- dplyr::rename(derivations, 'estimate' = dplyr::all_of(estimate), 'longitude' = 'x', 'latitude' = 'y')

  # unique observation code
  derivations$id <- frame$id

  # year
  derivations$year <- as.integer(year)

  return(list(derivations))

}

<br>

## Density Estimates

### The Experiments

The list of experiment files

In [3]:
files <- list.files(path = file.path(getwd(), 'warehouse', 'ESPEN', 'experiments', 'extended'),
                    full.names = TRUE)

<br>

A sample file

In [4]:
frame <- read.csv(file = files[3])
str(frame)

'data.frame':	357 obs. of  42 variables:
 $ iso3            : chr  "BDI" "BDI" "BDI" "BDI" ...
 $ iso2            : chr  "BI" "BI" "BI" "BI" ...
 $ admin1_id       : int  706 715 702 717 710 710 709 717 705 705 ...
 $ admin2_id       : int  6389 6414 6378 6418 6402 6402 6396 6419 6387 6387 ...
 $ iu_id           : int  6389 6414 6378 6418 6402 6402 6396 6419 6387 6387 ...
 $ location        : chr  "bitare" "buhiga ii" "buhina" "buhonga" ...
 $ site_id         : int  NA NA 10243 NA NA 10373 NA NA NA 10314 ...
 $ longitude       : num  29.3 29.8 29.4 30.3 30.1 ...
 $ latitude        : num  -2.88 -2.91 -3.45 -3.65 -2.69 ...
 $ georeliability  : int  1 3 1 1 1 1 3 1 3 3 ...
 $ location_type   : chr  "school" "school" "" "school" ...
 $ survey_type     : chr  "mapping" "mapping" "" "mapping" ...
 $ year            : int  2014 2016 2014 2014 2014 2014 2014 2014 2014 2014 ...
 $ age_start       : int  12 13 NA 12 12 NA 13 12 12 NA ...
 $ age_end         : int  14 14 NA 16 15 NA 14 16 15 NA ..

<br>

### Experiment Co&ouml;rdinates & Density Estimates


Arguments

In [5]:
years <- seq(from = 2000, to = 2020, by = 5)
root <- file.path(getwd(), 'data', 'population')
affix <- '_30s.tif'

<br>

Next, extracting population density estimates, per map year, from the 

* [`geodata`](https://github.com/rspatial/geodata#data): The `geodata` population density maps are [accessible directly](https://geodata.ucdavis.edu/geodata/pop/), and are the Gridded Population of the World maps.
* [Gridded Populations of the World](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4/documentation)

Map year estimates only exist for the years $2000,2005, 2010, 2015, 2020$

In [6]:
estimates <- mapply(FUN = temporary, year = years, MoreArgs = list(root = root, affix = affix, frame = frame))
estimates <- dplyr::bind_rows(dplyr::all_of(estimates))
row.names(estimates) <- NULL

In [7]:
str(estimates)

'data.frame':	1785 obs. of  5 variables:
 $ estimate : num  203 392 808 132 381 ...
 $ longitude: num  29.3 29.8 29.4 30.3 30.1 ...
 $ latitude : num  -2.88 -2.91 -3.45 -3.65 -2.69 ...
 $ id       : int  0 1 2 3 4 5 6 7 8 9 ...
 $ year     : int  2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 ...


<br>

### Interpolations

A set of observations that share the same longitude/latitude co&ouml;rdinates 

In [8]:
partition <- estimates[estimates$id == 0, c('year', 'estimate')]

<br>

Missing years

In [9]:
missing <- seq(from = 2000, to = 2020)
missing <- missing[!(missing %in% years)]

<br>

Interpolation

In [10]:
extra <- stats::spline(x = partition$year, y = partition$estimate, method = 'natural', xout = missing)
extra <- data.frame(extra)
extra <- dplyr::rename(extra, 'year' = 'x', 'estimate' = 'y')
extra$year <- as.integer(extra$year)

<br>

Collating

In [11]:
extra <- rbind(partition[, c('year', 'estimate')], extra)
extra

,year,estimate
,<int>,<dbl>
1,2000,202.8000
358,2005,219.6000
715,2010,237.9000
1072,2015,257.7000
1429,2020,279.2000
17,2001,206.0976
2,2002,209.4108
3,2003,212.7552
4,2004,216.1464


<br>

Arranging

In [12]:
extra <- extra[with(extra, order(year, estimate)), ]
extra

,year,estimate
,<int>,<dbl>
1,2000,202.8000
17,2001,206.0976
2,2002,209.4108
3,2003,212.7552
4,2004,216.1464
358,2005,219.6000
5,2006,223.1280
6,2007,226.7280
7,2008,230.3940
